# 데이터 수집(네이버 기사, 댓글 크롤링)

In [ ]:
# 필요한 모듈 설치
!pip install nest-asyncio
!pip install --upgrade lxml
!pip install --upgrade pymongo
from IPython.display import clear_output
clear_output()

In [1]:
from IPython.display import clear_output
clear_output()

In [2]:
# 라이브러리 불러오기
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import asyncio
import json
import nest_asyncio
from pymongo import MongoClient
nest_asyncio.apply()

In [3]:
# 함수 정의

def get_comments(refer_url, comment_url) : # 댓글 목록을 json 형태로 받아오는 함수
    comments = []
    next = None
    # 처음엔 댓글 개수를 모르므로 충분히 큰 수를 넣어 줌
    comment_count = 10e6
    headers = {
        'User-Agent' : '.......',
        'referer': refer_url
    }

    # 수집한 댓글 수가 첫번째에 수집한 총 댓글 수 보다 많다면 반복을 종료합니다.
    while len(comments) < comment_count :
        comment_url_next = comment_url + '&moreParam.next=' + next if next else comment_url
        res = requests.get(comment_url_next, headers=headers)
        dic = json.loads(res.text[res.text.index('(')+1:-2])
        comments.extend(list(map(lambda x : {
            'id': x['commentNo'],
            'time': x['regTime'], 
            'uid': x['idNo'], 
            'text': x['contents'], 
            'sympathy': x['sympathyCount'], 
            'antipathy': x['antipathyCount'] 
            }, dic['result']['commentList'])))
        comment_count = dic['result']['count']['comment']
        next = dic['result']['morePage']['next'] if comment_count else None
    # 필터로 삭제된 댓글을 걸러줍니다
    comments=list(filter(lambda x: len(x['text']), comments))
    return comments



In [4]:
def get_article(offer, article_url) : # 뉴스 기사를 html 형태로 받아오는 함수
    article = {}
    headers = {'User-Agent' : '........'}
    res = requests.get(article_url, headers=headers)
    if res.url != article_url :
        return None
    soup = BeautifulSoup(res.text, 'lxml')
    article['offer'] = offer
    article['url'] = article_url
    article['title'] = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_title > h2').text
    article['text'] = soup.select_one('#dic_area').text
    article['date'] = soup.select_one('#ct > div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span.media_end_head_info_datestamp_time').attrs['data-date-time']
    return article




In [5]:
async def main(filename, start, end) : # 위 함수를 이용해 데이터를 받아오고 저장하는 함수
    def get_data(offer, date, oid, aid, count, max) :
        try :
            # article_url: 원본 기사 주소
            # refer_url: 댓글 보기를 누르면 나오는 댓글 페이지 주소
            # comment_url: 네트워크 탭에서 확인 가능한 동적으로 생성되는 주소
            article_url = f'.....'
            refer_url = f'.....'
            comment_url = f'.....'
            article = get_article(offer, article_url)
            # article이 None이 반환되는 경우는 리다이렉트가 발생하는 경우
            if article is None : return None
            comments = get_comments(refer_url, comment_url)
            # 댓글 수가 0개인 기사를 어떻게 할지 추후에 결정해야 함
            article['comments'] = comments
            print(date, str(count) + '/' + str(max), '댓글 개수:' + str(len(comments)))
            return article
        except :
            # 왜 오류가 뜨는지는 모르겠으나, 다시 실행하면 되는걸로 봐서 한번에 너무 많은 요청을 하는게 문제인 듯
            # 로직을 안전하게 고치기 귀찮으니 오류가 뜬 url을 로그에 남겨두고 추후에 추가하자.
            errors.append([date, oid, aid])
            print(date, article_url, '오류 발생')
            return None

    # 매개변수로 받은 시작, 종료일자로 기간 배열을 만들어줌
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=d)).strftime('%Y%m%d') for d in range((end-start).days+1)]

    # 파일 불러오기
    with open(filename, 'r', encoding = 'utf-8') as f :
        data = json.load(f)

    # db 연결
    client = MongoClient(
        host='.....', 
        port=....,
        username='....',
        password='....',
        authSource='..')
    db = client['...']
    col = db['....']

    loop = asyncio.get_event_loop()
    for date in dates :
        
        # 기존 일자의 기사들 삭제
        # d = datetime.strptime(date, "%Y%m%d")
        # d = datetime.strftime(d, "%Y-%m-%d")
        # col.delete_many({'date': {"$regex" : d}})

        # 카운트 변수를 0으로 초기화
        count = 0
        max = len(data[date])
        futures = []
        errors = []
        for offer, oid, aid in data[date] :
            count += 1
            futures.append(loop.run_in_executor(None, get_data, offer, date, oid, aid, count, max))
            if count % 100 == 0 or count == len(data[date]) :
                # 백번째마다 플러시
                articles = await asyncio.gather(*futures)
                articles = list(filter(lambda x : x is not None, articles))
                # 디비에 데이터 전송
                col.insert_many(articles)
                # 오류 뜬 기사들은 로그 파일에 넣기
                with open('log.txt', 'a') as f :
                    for i in errors :
                        f.write(' '.join(i) + '\n')        
                clear_output(wait=True)
                print(len(articles), '완료', len(errors), '에러', str(count) + '/' + str(len(data[date])))
                futures = []
                errors = []

In [6]:
%%time
 
# main(파일 이름, 시작 날짜, 종료 날짜)
asyncio.run(main('2021.json', '20210329', '20210331'))
# 도중에 멈추거나 해서 중단한 경우 밑 출력창을 보고 중지된 날짜를 다시 시작 날
# 
# 
# 짜에 기입하여 실행해주면 됨.

43 완료 0 에러 2343/2343
Wall time: 34min 16s
